Transform video to audio

In [ ]:
from audio_extractor import extract_audio

debra_video = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Soundtrack/[ONTIVA.COM] Debra-144p.mp4"
debra_audio_file = "debra_song"

baby_driver_video_file = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Baby.Driver.2017.720p.BluRay.x264-[YTS.AG].mp4"
baby_driver_audio_file = "baby_driver_audio"

stw_movie = "C:/Users/iggym/Documents/Movies/Star Wars Episode V - The Empire Strikes Back (1980) [BluRay] [720p] [YTS.AM]/Star.Wars.Episode.V.-.The.Empire.Strikes.Back.1980.720p.BluRay.x264-[YTS.AM].mp4"
stw_audio = "starwars"

sample_rate = 22000

# extract_audio(debra_video, sample_rate, debra_audio_file)
# extract_audio(baby_driver_video_file, sample_rate, baby_driver_audio_file)
extract_audio(stw_movie, sample_rate, stw_audio)

C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
Audio extracted: C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
starwars.22000.wav
STARTING: ffmpeg -i C:/Users/iggym/Documents/Movies/Star Wars Episode V - The Empire Strikes Back (1980) [BluRay] [720p] [YTS.AM]/Star.Wars.Episode.V.-.The.Empire.Strikes.Back.1980.720p.BluRay.x264-[YTS.AM].mp4 -ac 1 -ar 22000 starwars.22000.wav


'starwars.22000.wav'

Generate Descriptors

In [ ]:
from audio_descriptor_generator import create_audio_descriptors

sample_rate = 22000
descriptors_per_second = 2
dimension = 32
window = int(sample_rate / descriptors_per_second)
hop = window

baby_driver_audio_file = "baby_driver_audio.22000.wav"
debra_song_audio_file = "debra_song.22000.wav"
stw_audio_file = "starwars.22000.wav"
"""
debra_descriptors = create_audio_descriptors(debra_song_audio_file, sample_rate, dimension, window, hop)
print(debra_descriptors.shape)
baby_descriptors = create_audio_descriptors(baby_driver_audio_file, sample_rate, dimension, window, hop)
print(baby_descriptors.shape)
"""
stw_descriptors = create_audio_descriptors(stw_audio_file, sample_rate, dimension, window, hop)
print(stw_descriptors.shape)
"""
baby_driver_descriptors_file = f"baby_driver_audio_descriptors_{descriptors_per_second}.bin"
debra_song_descriptors_file = f"debra_song_descriptors_{descriptors_per_second}.bin"
"""
stw_descriptors_file = f"stw_descriptors_{descriptors_per_second}.bin"
"""
debra_descriptors.tofile(debra_song_descriptors_file, sep="\n")
baby_descriptors.tofile(baby_driver_descriptors_file, sep="\n")
"""
stw_descriptors.tofile(stw_descriptors_file, sep="\n")
"""
song_shape = debra_descriptors.shape
movie_shape = baby_descriptors.shape
"""
stw_shape = stw_descriptors.shape

1.12 seconds
(3434, 32)
(15284, 32)


Calculate distances

In [14]:
import numpy
import time
from scipy.spatial import distance

descriptors_per_second = 2
"""
song_shape = (687, 32)
"""
movie_shape = (13520, 32)
stw_shape = (15284, 32)

# baby_driver_descriptors_file = f"baby_driver_audio_descriptors_{descriptors_per_second}.bin"
# debra_song_descriptors_file = "debra_song_descriptors.bin"
# debra_song_descriptors_file = f"debra_song_descriptors_{descriptors_per_second}.bin"

# stw_descriptors_file = f"drive/MyDrive/RIM/proj/stw_descriptors_{descriptors_per_second}.bin"
stw_descriptors_file = f"stw_descriptors_{descriptors_per_second}.bin"

debra_song_shape = (687, 32)

# song_descriptors = numpy.fromfile(debra_song_descriptors_file, sep="\n").reshape(debra_song_shape)
# print(f"Debra {song_descriptors.shape}")
# print(song_descriptors[10:-10].shape)

# movie_shape = movie_shape
stw_descriptors = numpy.fromfile(stw_descriptors_file, sep="\n").reshape(stw_shape)
print(f"Starwars {stw_descriptors.shape}")

# movie_descriptors = numpy.fromfile(baby_driver_descriptors_file, sep="\n").reshape(movie_shape)
# print(f"Baby {movie_descriptors.shape}")



t0 = time.time()
distances = distance.cdist(stw_descriptors, stw_descriptors)
# distances = distance.cdist(movie_descriptors, movie_descriptors)
t1 = time.time()

print(f"Distances {distances.shape} {round(t1-t0, 2)} secs")

Starwars (15284, 32)
Distances (15284, 15284) 18.43 secs


Define neighbour function

In [15]:
def add_offset_to_index(offset_threshold, index, offset):
    if index >= offset_threshold:
        return index + (offset*2)
    return index

def get_closest_neighbours(song_index, num_of_neighs, distance_matrix, min_offset):
    offset_range = ((song_index - min_offset) % distance_matrix.shape[1], (song_index + min_offset) % distance_matrix.shape[1])
    switch = True
    
    if offset_range[0] < offset_range[1]:
        start_neighs = distance_matrix[song_index][:offset_range[0]]
        end_neighs = distance_matrix[song_index][offset_range[1]:]
        new_neighs = numpy.append(start_neighs, end_neighs)
    else:
        new_neighs = distance_matrix[song_index][offset_range[1]:offset_range[0]]

    results = numpy.argpartition(new_neighs, num_of_neighs)[:num_of_neighs]
    new_results = []
    for i in range(len(results)):
        if switch:
            offset_threshold = offset_range[1]
            new_results.append(add_offset_to_index(offset_threshold, results[i], min_offset_descriptors))
        else:
            offset_threshold = offset_range[0]
            new_results.append(add_offset_to_index(offset_threshold, results[i], min_offset_descriptors))

    return numpy.array(new_results)

Get neighbours

In [16]:
from duplicate_searcher import Neighbours
import time

number_of_neighbours = 10
min_offset_seconds = 60
min_offset_descriptors = min_offset_seconds * descriptors_per_second

neighbours = []
total = distances.shape[0]
t10 = time.time()
for i in range(distances.shape[0]):
    print(f"Progress: {round((i/total)*100, 2)}%", end="\r")
    """
    song_descriptor = distances[i]
    neighs = numpy.argpartition(song_descriptor, number_of_neighbours)[:number_of_neighbours]
    """
    neighs = get_closest_neighbours(i, number_of_neighbours, distances, min_offset_descriptors)
    neighbours.append(neighs)
t11 = time.time()
neighbours = Neighbours(numpy.array(neighbours))
print(f"Neighbours {neighbours.shape()} {round(t11-t10, 2)} secs")

Neighbours (15284, 10) 8.5 secs


Create Candidates

In [17]:
from duplicate_searcher import Candidate

candidates = []
song_indexes = range(distances.shape[0])
total_songs = distances.shape[0]
for song in song_indexes:
    print(f"Progress {round((song/total_songs)*100, 2)}%", end="\r")
    for neighbour in neighbours.search(song):
        candidates.append(Candidate(song, neighbour, descriptors_per_second))

print(f" Candidates {len(candidates)}")


 Candidates 152840


Find sequence

In [18]:
copies = []

max_missing_streak_secs = 1
max_missing_streak = max_missing_streak_secs * descriptors_per_second
min_duration_seconds = 3
min_duration = min_duration_seconds * descriptors_per_second

total_candidates = len(candidates)
for i in range(len(candidates)):
    print(f"Progress {round((i/total_candidates)*100, 2)}%", end="\r")
    cand = candidates[i]
    current_candidate = cand.find_next(neighbours, max_missing_streak, min_duration)
    if current_candidate.sequence_duration >= min_duration and current_candidate.score() >= 1:
        copies.append(current_candidate)

print(len(copies))   
    

32


Contain

In [19]:
filtered = []

for i in range(len(copies)):
    print(f"Progress {round((i/len(copies))*100, 2)}%", end="\r")
    cani = copies[i]
    add = True
    for j in range(len(copies)):
        canj = copies[j]
        
        if i == j:
            continue
        
        elif cani.contains(canj):
            add = False
            break
    
    if add:
        filtered.append(cani)

print(len(filtered))


32


Sort

In [20]:
# sorted_candidates = filtered
sorted_candidates = sorted(filtered, key=lambda c: c.song_descriptor_index)

print(len(sorted_candidates))

32


Combine

In [21]:
max_offset_secs = 2
max_offset = max_offset_secs * descriptors_per_second
max_combine_dist_secs = 3
max_combine_distance = max_combine_dist_secs * descriptors_per_second

for i in range(len(sorted_candidates)):
    print(f"Progress {round((i/len(sorted_candidates))*100, 2)}%", end="\r")
    copy_i = sorted_candidates[i]
    for j in range(i + 1, len(sorted_candidates)):
        copy_j = sorted_candidates[j]

        if (copy_i.distance(copy_j) <= max_combine_distance) and (copy_i.offset_diff(copy_j) <= max_offset):
            copy_i.combine(copy_j)

print(len(sorted_candidates))

32


Overlapped

In [22]:
repeated = set()
off_set_diff_limit_secs = 10
off_set_diff_limit = off_set_diff_limit_secs * descriptors_per_second
max_overlap_dist_secs = 3
max_overlap_distance = max_overlap_dist_secs * descriptors_per_second

for i in range(len(sorted_candidates)):
    copy_i = sorted_candidates[i]
    for j in range(i + 1, len(sorted_candidates)):
        copy_j = sorted_candidates[j]

        if copy_i.distance(copy_j) <= max_overlap_distance and copy_i.offset_diff(copy_j) <= off_set_diff_limit_secs:
            if copy_i.sequence_duration >= copy_j.sequence_duration:
                repeated.add(copy_j)
            else:
                repeated.add(copy_i)

for copy in repeated:
    sorted_candidates.remove(copy)

print(len(sorted_candidates))

18


Delete short copies

In [23]:
filtered_copies = []

for copy in sorted_candidates:
    if copy.sequence_duration > min_duration:
        filtered_copies.append(copy)

print(len(filtered_copies))

6


Get Average Distances

In [24]:
for match in filtered_copies:
    match.avg_distance(distances)

sorted_and_filtered = sorted(filtered_copies, key=lambda c: c.avg_distance)

Show Sequences

In [28]:
import datetime

# print("Song Seconds      | Movie Seconds     | Avg Distance | Score")
# print("------------------------------------------------------------")

# Debra starts in aprox. 0:41:30
add_cut = 0

# It worked, the second and third sequence work, 2:01:12 - 2:01:16 | 2:01:48 - 2:01:52 star wars

from prettytable import PrettyTable 
  
# Specify the Column Names while initializing the Table 
myTable = PrettyTable(["Song Seconds", "Movie Seconds", "Avg Distance", "Score"])

for match in sorted_and_filtered:
    song_start_secs = str(datetime.timedelta(seconds=int((match.song_descriptor_index + add_cut) / descriptors_per_second)))
    song_end_secs = str(datetime.timedelta(seconds=int((match.song_end_index() + add_cut) / descriptors_per_second)))
    movie_start_secs = str(datetime.timedelta(seconds=int((match.movie_descriptor_index + add_cut) / descriptors_per_second)))
    movie_end_secs = str(datetime.timedelta(seconds=int((match.movie_end_index() + add_cut) / descriptors_per_second)))
    avg_distance = match.avg_distance    

    # print(f"{match.song_descriptor_index} - {match.song_end_index()} | {match.movie_descriptor_index} - {match.movie_end_index()}")
    myTable.add_row([f"{song_start_secs} - {song_end_secs}", f"{movie_start_secs} - {movie_end_secs}", avg_distance, match.score()])
    # print(f"{song_start_secs} - {song_end_secs} | {movie_start_secs} - {movie_end_secs} | {avg_distance} | {match.score()}")

print(myTable)

+-------------------+-------------------+--------------+-------+
|    Song Seconds   |   Movie Seconds   | Avg Distance | Score |
+-------------------+-------------------+--------------+-------+
| 0:15:05 - 0:15:09 | 0:10:09 - 0:10:13 |    36.66     |  1.0  |
| 1:45:15 - 1:45:19 | 0:54:23 - 0:54:27 |    52.74     |  1.5  |
| 0:18:47 - 0:18:51 | 0:41:36 - 0:41:40 |    70.23     |  4.0  |
| 1:24:40 - 1:24:45 | 1:11:47 - 1:11:52 |    77.73     |  1.5  |
| 0:46:08 - 0:46:12 | 0:42:26 - 0:42:30 |    92.07     |  4.0  |
| 0:12:26 - 0:12:30 | 0:12:06 - 0:12:10 |    191.59    |  2.0  |
+-------------------+-------------------+--------------+-------+
